In [2]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math

import wmi

In [10]:
# Initialize camera
w_cam, h_cam = 1280, 700

cap = cv2.VideoCapture(0)
cap.set(3, w_cam)  # sets width
cap.set(4, h_cam)  # sets height

pTime = 0

# Initialize hand detector
detector = htm.handDetector(detectionCon=0.7)

# Initialize WMI for brightness control
c = wmi.WMI(namespace='wmi')

# Initialize variables for brightness bar
bar_x = 30  # X-coordinate of the brightness bar
bar_y = 50  # Y-coordinate of the brightness bar
bar_height = 300  # Height of the brightness bar
bar_width = 25  # Width of the brightness bar
bar_pos = 50 # initial position
bar_color = (0,0,255) # initial color

while True:
    success, img = cap.read()
    if not success:
        break

    img = cv2.flip(img, 1)  # Flip the image horizontally

    # Find hands and landmarks
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)

    if lmList:
        # Get the tip positions of the thumb and index finger
        x1, y1 = lmList[4][1], lmList[4][2]  # Thumb tip
        x2, y2 = lmList[8][1], lmList[8][2]  # Index finger tip

        # Calculate the midpoint between the thumb and index finger
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # Draw circles and line
        cv2.circle(img, (x1, y1), 15, (0, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (0, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 0), 3)
        cv2.circle(img, (cx, cy), 7, (255, 0, 0), 4)

        # Calculate length between thumb and index finger
        length = math.hypot(x2 - x1, y2 - y1)

        # Map length to brightness range
        brightness = np.interp(length, [50, 200], [0, 100])

        # Set monitor brightness using WMI
        try:
            brightness = int(brightness)  # Convert to integer (0-100)
            c.WmiMonitorBrightnessMethods()[0].WmiSetBrightness(brightness, 0)  # Set brightness
        except Exception as e:
            print(f"Error setting brightness: {e}")

        # Update bar position based on brightness
        bar_pos = int(np.interp(brightness, [0, 100], [bar_y, bar_y + bar_height - bar_width]))

        # Calculate bar color based on brightness
        bar_color = (0, 255, 0)  # Default color is green
        if brightness < 20:
            bar_color = (0, 0, 255)  # Red color for low brightness levels
        elif brightness < 50:
            bar_color = (0, 165, 255)  # Orange color for mid-low brightness levels
        elif brightness < 80:
            bar_color = (0, 255, 255)  # Yellow color for mid-high brightness levels

        # Display brightness percentage
        cv2.putText(img, f'Brightness: {int(brightness)}%', (600, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

    # Draw brightness bar
    cv2.rectangle(img, (bar_x, bar_y), (bar_x + bar_width, bar_y + bar_height), (0, 0, 255), 3)
    cv2.rectangle(img, (bar_x, bar_pos), (bar_x + bar_width, bar_pos + bar_width), bar_color, cv2.FILLED)

    # Calculate FPS
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    # Display FPS
    cv2.putText(img, f'FPS: {int(fps)}', (40, h_cam - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)

    # Show the image with annotations
    cv2.imshow("Brightness Control", img)

    # Exit loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()
